In [ ]:
import pandas as pd
import plotly.express as px
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from xgboost import XGBClassifier

In [ ]:
train = pd.read_csv("../data/split/train.csv")
train.shape, train.keys()

In [ ]:
fig = px.histogram(train, x="Class")
fig.update_layout(
    autosize=False,
    width=800,
    height=400,
)
fig

In [ ]:
y_train = train[["Class"]]
X_train = train.drop(["Class"], axis=1)

labels = ["Not Fraud", "Fraud"]
model = XGBClassifier()
k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
train_preds = cross_val_predict(model, X_train, y_train, cv=k_fold)
print(classification_report(y_train, train_preds, target_names=labels))
ConfusionMatrixDisplay.from_predictions(y_train, train_preds)